In [1]:
%pylab
import pandas as pd
import pickle
import os
print('complete')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
complete


In [2]:
# Task: find sequence of tokens where BSC-seq-IF gets it right but MV and HMMCrowd fail.
# Steps:
# 1. load gold
# 2. load text
# 3. load crowd annotations
# 3. load predictions from each method.
# 4. Scroll through gold, stop when we get a token where BSC is correct and others are wrong.
# 5. Map the token idxs to NER labels or IOB labels.
# 6. For the 10 previous and subsequent tokens, print:
# (a) the text, 
# (b) the crowd labels 
# (c) the gold and 
# (d) the predictions
# 8. ask user to save it. If no, continue scrolling to next candidate.
# 7. Use inkscape to turn this into a pretty diagram

In [7]:
golddir = '../../../data/bayesian_annotator_combination/data/ner/'

goldfile = os.path.join(golddir, 'task1_test_gt.csv')
gold = pd.read_csv(goldfile, header=None).values
print(gold.shape)

annofile = os.path.join(golddir, 'task1_test_annos.csv')
annos = pd.read_csv(annofile, header=None).values
print(annos.shape)

textfile = os.path.join(golddir, 'task1_test_text.csv')
text = pd.read_csv(textfile, header=None).values
print(text.shape)

(42975, 1)
(42975, 47)
(42975, 1)


In [21]:
resdir = '../../../data/bayesian_annotator_combination/output/good_results/ner-by-sentence/'
resfile = os.path.join(resdir, 'pred_started-2018-07-04-02-50-58-Nseen6045.csv') # 06-23-07-01-12-Nseen6045.csv')
res = pd.read_csv(resfile)
print(res.columns)
print(res.shape)
bronze = [] # list of runner up methods' predictions
bronze.append(res[" 'bac_seq_integrateBOF_integrateLSTM'"].values.astype(int))

#bronze.append(res["# 'majority'"].values)
#bronze.append(res[" 'ds'"].values)

#resfile = os.path.join(resdir, 'pred_started-2018-06-24-07-48-06-Nseen6045.csv')
#res = pd.read_csv(resfile)
#print(res.columns)
#bronze.append(res["# 'HMM_crowd'"].values)
#bronze.append(res[" 'HMM_crowd_then_LSTM'"].values)

#resfile = os.path.join(resdir, 'pred_started-2018-06-23-05-20-51-Nseen6045.csv')
#res = pd.read_csv(resfile)
print(res.columns)
silver = res["# 'bac_seq_integrateBOF'"].values.astype(int)

Index(['# 'bac_seq_integrateBOF'', ' 'bac_seq_integrateBOF_integrateLSTM'',
       ' 'bac_seq_integrateBOF_integrateLSTM_atEnd'',
       ' 'bac_seq_integrateBOF_then_LSTM''],
      dtype='object')
(82411, 4)
Index(['# 'bac_seq_integrateBOF'', ' 'bac_seq_integrateBOF_integrateLSTM'',
       ' 'bac_seq_integrateBOF_integrateLSTM_atEnd'',
       ' 'bac_seq_integrateBOF_then_LSTM''],
      dtype='object')


In [ ]:
for i in range(len(gold)):
    
    matchfound = False
    
    if gold[i] == silver[i]:
        for b in bronze:
            if gold[i] != b[i]:
                print('Interesting data found. Type s to save, n to continue, x to exit: ')
                matchfound = True
                break

    if not matchfound:
        continue
    
    start = i - 5
    if start < 0:
        start = 0

    end = i + 5
    if end > len(gold):
        end = len(gold)

    live_workers = np.argwhere(np.any(annos[start:end] != -1, axis=0)).flatten()
    
    print('Index = %i to %i' % (start, end))
    
    for worker in live_workers:
        print(annos[start:end, worker].T)
    print(text[start:end].flatten())
    print(gold[start:end].flatten())
    print(silver[start:end].flatten())
    for b in bronze:
        print(b[start:end].flatten())

    input_data = input(">>>")
    print(input_data)
    
    if input_data == 'x':
        break
    if input_data == 's':
        
        with open('../error_analysis_method_comparison.txt', 'a') as fh:
            
            for worker in live_workers:
                fh.write(str(annos[start:end, worker].T))
                fh.write('\n')
            
            fh.write(str(text[start:end].flatten()))
            fh.write('\n')
            fh.write(str(gold[start:end].flatten()))
            fh.write('\n')
            fh.write(str(silver[start:end].flatten()))
            fh.write('\n')

            for b in bronze:
                fh.write(str(b[start:end].flatten()))
                fh.write('\n')
            fh.write('\n')
                    

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 320 to 330
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['among' 'those' 'called' '.' 'But' 'Currin' ',' 'who' 'is' 'advising']
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 433 to 443
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['amnesty' 'application' 'before' 'that' '.' 'Currin' 'said' 'the' 'law'
 'had']
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 470 to 480
[1 1 1 1 1 1 1 1 1 1]
[2 1 1 1 1 2 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 2 0 0 0 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['ANC' ',' 'de' 'Klerk' "'s" 'National' 'Party' 'and' 'the' 'right-wing']
[2 1 4 3 1 2 0 1 1 1]
[1 1 1 1 1 2 0 1 1 1]

>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 3203 to 3213
[ 1  4  1 -1 -1 -1 -1 -1 -1 -1]
[1 1 1 8 1 1 1 1 1 1]
[-1 -1 -1  1  1  1  1  1  1  1]
[ 1  4  1 -1 -1 -1 -1 -1 -1 -1]
[ 1  4  1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1  1  1  1  1  1  1  1]
[1 1 1 8 1 1 1 1 1 1]
[-1 -1 -1  1  1  8  1  1  1  1]
[',' 'Salisbury' '17-0-91-0' 'SOCCER' '-' 'ENGLISH' 'LEAGUE' 'RESULTS'
 'STANDINGS' '.']
[1 4 1 1 1 8 1 1 1 1]
[1 4 1 8 1 8 1 1 1 1]
[1 4 1 8 1 1 1 1 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 3222 to 3232
[1 1 1 1 6 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 6 6 1 6 1 1]
[1 1 1 1 2 0 1 2 1 1]
[8 1 1 1 2 0 1 2 1 1]
['Sunday' ':' 'Premier' 'league' 'Manchester' 'United' '2' 'Blackburn' '2'
 'Standings']
[1 1 1 1 2 0 1 2 1 1]
[1 1 1 1 2 0 1 2 1 1]
[1 1 1 1 2 1 1 2 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 3282 to 3292
[1 1 1 1 6 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 6 6 1 1 1 1]